In [10]:
import numpy as np

### Open Words with BIO Notation Label

In [132]:
import json
def get_BIO_data(filename):
    with open(filename) as f:
        labeled = json.load(f)

    display(type(labeled))
    data = []
    for sentence in labeled:
        sen = []
        for item in sentence:
            tup = (item['token'],item['pos'],item['label'])
            sen.append(tup)
        data.append(sen)
    return data

def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    features = [
        'bias',
        'word.lower=' + word.lower(),
        'postag=' + postag
    ]
    # Fitur untuk kata non awal kalimat
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS') # Tanda awal kalimat
    # Fitur untuk kata non akhir kalimat
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS') #tanda akhir kalimat
    return features

def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [label for (token, postag, label) in doc]

def get_split_index(X,n_split):
    kf = KFold(n_splits=n_split)
    train_split = []
    test_split = []
    for train_index, test_index in kf.split(X):
        train_split.append(train_index)
        test_split.append(test_index)
    return train_split,test_split

In [135]:
import pycrfsuite
def generate_model(X,y,train_split,test_split):
    trainer = pycrfsuite.Trainer(verbose=True)
    for i in range(len(train_split)):
        X_train = [X[j] for j in train_split[i]]
        y_train = [y[j] for j in train_split[i]]
        for xseq, yseq in zip(X_train, y_train):
            trainer.append(xseq, yseq)
        trainer.set_params({
            'c1': 0.1, #L1 penalty
            'c2': 0.01, #L2 penalty
            'max_iterations': 200,
            'feature.possible_transitions': True
        })
        trainer.train('crf.model_'+str(i))

In [136]:
data = get_BIO_data('reviews-aspect.json')
X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]
train_split, test_split = get_split_index(X,10)

#Generate Model with pycrfsuite
generate_model(X,y,train_split,test_split)

list

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 5143
Seconds required: 0.015

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 2965.432524
Feature norm: 1.000000
Error norm: 2045.092997
Active features: 5054
Line search trials: 1
Line search step: 0.000126
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 2537.814600
Feature norm: 1.345533
Error norm: 717.591510
Active features: 4811
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 2433.836074
Feature norm: 1.471054
Error norm: 658.487876
Active features: 4984
Line search trials: 1
Line search step: 1.000000
Seconds required for this iterat

***** Iteration #74 *****
Loss: 423.797137
Feature norm: 61.771441
Error norm: 11.528746
Active features: 1482
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #75 *****
Loss: 423.654652
Feature norm: 61.736057
Error norm: 5.419477
Active features: 1482
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #76 *****
Loss: 423.540157
Feature norm: 61.773770
Error norm: 8.944973
Active features: 1489
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #77 *****
Loss: 423.422176
Feature norm: 61.761684
Error norm: 4.824096
Active features: 1486
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #78 *****
Loss: 423.329004
Feature norm: 61.797866
Error norm: 10.923947
Active features: 1486
Line search trials: 1
Line search step: 1.000000
Seconds required for this i

***** Iteration #147 *****
Loss: 420.281570
Feature norm: 62.830174
Error norm: 5.906312
Active features: 1377
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #148 *****
Loss: 420.280866
Feature norm: 62.844584
Error norm: 10.379672
Active features: 1375
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #149 *****
Loss: 420.225325
Feature norm: 62.841522
Error norm: 4.802472
Active features: 1370
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #150 *****
Loss: 420.209690
Feature norm: 62.847791
Error norm: 7.787228
Active features: 1368
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #151 *****
Loss: 420.184433
Feature norm: 62.844843
Error norm: 6.277330
Active features: 1369
Line search trials: 1
Line search step: 1.000000
Seconds required for th

***** Iteration #197 *****
Loss: 419.391296
Feature norm: 62.765699
Error norm: 2.864041
Active features: 1364
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #198 *****
Loss: 419.381434
Feature norm: 62.765037
Error norm: 4.660118
Active features: 1363
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.006

***** Iteration #199 *****
Loss: 419.369950
Feature norm: 62.760912
Error norm: 4.081389
Active features: 1361
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.005

***** Iteration #200 *****
Loss: 419.359413
Feature norm: 62.763702
Error norm: 5.272094
Active features: 1360
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.008

L-BFGS terminated with the maximum number of iterations
Total seconds required for training: 0.738

Storing the model
Number of active features: 1360 (5143)
Number of active attributes

...

***** Iteration #136 *****
Loss: 1583.306442
Feature norm: 127.089255
Error norm: 42.630728
Active features: 2041
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #137 *****
Loss: 1582.880941
Feature norm: 127.104672
Error norm: 21.532114
Active features: 2040
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.035

***** Iteration #138 *****
Loss: 1582.794763
Feature norm: 127.175077
Error norm: 42.039815
Active features: 2036
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.026

***** Iteration #139 *****
Loss: 1582.380204
Feature norm: 127.199374
Error norm: 27.432367
Active features: 2037
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.028

***** Iteration #140 *****
Loss: 1582.341856
Feature norm: 127.288709
Error norm: 50.957675
Active features: 2035
Line search trials: 1
Line search step: 1.000000
Seconds r

0....1....2....3....4....5....6....7....8....9....10
Number of features: 5607
Seconds required: 0.085

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 30881.087112
Feature norm: 1.000000
Error norm: 20135.751497
Active features: 5504
Line search trials: 1
Line search step: 0.000013
Seconds required for this iteration: 0.051

***** Iteration #2 *****
Loss: 26791.639508
Feature norm: 1.334027
Error norm: 7332.367457
Active features: 5276
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.027

***** Iteration #3 *****
Loss: 25746.440084
Feature norm: 1.456631
Error norm: 6863.796690
Active features: 5406
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.029

***** Iteration #4 *****
Loss: 22277.498757
Feature norm: 1.811149
Error norm: 5301.347651
Active 

***** Iteration #65 *****
Loss: 1782.500695
Feature norm: 124.592630
Error norm: 54.511785
Active features: 2319
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #66 *****
Loss: 1778.713626
Feature norm: 124.927127
Error norm: 73.900612
Active features: 2316
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #67 *****
Loss: 1775.231191
Feature norm: 125.085328
Error norm: 78.822377
Active features: 2307
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.026

***** Iteration #68 *****
Loss: 1771.798250
Feature norm: 125.406451
Error norm: 57.607050
Active features: 2298
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #69 *****
Loss: 1768.968409
Feature norm: 125.502819
Error norm: 41.921502
Active features: 2290
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #113 *****
Loss: 1708.125355
Feature norm: 127.354148
Error norm: 55.448918
Active features: 2106
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.032

***** Iteration #114 *****
Loss: 1707.552280
Feature norm: 127.378580
Error norm: 39.263655
Active features: 2103
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #115 *****
Loss: 1707.333969
Feature norm: 127.508979
Error norm: 67.172561
Active features: 2101
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.024

***** Iteration #116 *****
Loss: 1706.533371
Feature norm: 127.540722
Error norm: 35.997036
Active features: 2097
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #117 *****
Loss: 1706.297719
Feature norm: 127.651157
Error norm: 61.625149
Active features: 2097
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #182 *****
Loss: 1688.512211
Feature norm: 130.459788
Error norm: 18.275596
Active features: 2033
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #183 *****
Loss: 1688.475194
Feature norm: 130.474004
Error norm: 38.844611
Active features: 2034
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #184 *****
Loss: 1688.167489
Feature norm: 130.467016
Error norm: 15.685160
Active features: 2036
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.025

***** Iteration #185 *****
Loss: 1688.138106
Feature norm: 130.486741
Error norm: 37.448524
Active features: 2032
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.023

***** Iteration #186 *****
Loss: 1687.844268
Feature norm: 130.481258
Error norm: 18.729628
Active features: 2036
Line search trials: 1
Line search step: 1.000000
Seconds r

## Model Evaluation

In [131]:
from sklearn.metrics import classification_report

labels = {"B": 0, "I": 1,"O":2}
target_names = ['B', 'I','O']

for i in range(len(train_split)):
    print(i)
    tagger = pycrfsuite.Tagger()
    tagger.open('crf.model_'+str(i))
    X_test = [X[j] for j in test_split[i]]
    y_test = [y[j] for j in test_split[i]]
    y_pred = [tagger.tag(xseq) for xseq in X_test]
#     for iterr, X_test_item in enumerate(X_test):
#         for idx,word in enumerate(X_test_item):
#             print(word[1]+" - "+y_test[iterr][idx]+" - "+y_pred[iterr][idx])
    truth = np.array([labels[bio] for sentence in y_test for bio in sentence])
    prediction = np.array([labels[bio] for sentence in y_pred for bio in sentence])
    print(classification_report(truth, prediction, target_names=target_names))

0
              precision    recall  f1-score   support

           B       0.77      0.49      0.60        82
           I       0.55      0.12      0.20        48
           O       0.91      0.99      0.95       832

   micro avg       0.90      0.90      0.90       962
   macro avg       0.74      0.53      0.58       962
weighted avg       0.88      0.90      0.88       962

1
              precision    recall  f1-score   support

           B       0.89      0.89      0.89        45
           I       0.95      1.00      0.98        21
           O       0.99      0.99      0.99       722

   micro avg       0.99      0.99      0.99       788
   macro avg       0.95      0.96      0.95       788
weighted avg       0.99      0.99      0.99       788

2
              precision    recall  f1-score   support

           B       0.98      0.93      0.96        59
           I       1.00      1.00      1.00         9
           O       0.99      1.00      1.00       751

   micro avg  

In [129]:
i=1
tagger = pycrfsuite.Tagger()
tagger.open('crf.model_'+str(i))
X_test = [X[j] for j in test_split[i]]
y_test = [y[j] for j in test_split[i]]
y_pred = [tagger.tag(xseq) for xseq in X_test]
#     for iterr, X_test_item in enumerate(X_test):
#         for idx,word in enumerate(X_test_item):
#             print(word[1]+" - "+y_test[iterr][idx]+" - "+y_pred[iterr][idx])
truth = np.array([labels[bio] for sentence in y_test for bio in sentence])
prediction = np.array([labels[bio] for sentence in y_pred for bio in sentence])
print(prediction)
print(classification_report(truth, prediction, target_names=target_names))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 1 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 0
 1 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2
 0 1 2 2 2 2 2 2 2 2 2 2 

In [126]:
import pycrfsuite

zzz =2

trainer = pycrfsuite.Trainer(verbose=True)
X_train = [X[j] for j in train_split[zzz]]
y_train = [y[j] for j in train_split[zzz]]
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
trainer.set_params({
    'c1': 0.1, #L1 penalty
    'c2': 0.01, #L2 penalty
    'max_iterations': 200,
    'feature.possible_transitions': False
})
trainer.train('crf.model_coba')

from sklearn.metrics import classification_report

labels = {"B": 0, "I": 1,"O":2}
target_names = ['B', 'I','O']

tagger = pycrfsuite.Tagger()
tagger.open('crf.model_coba')
X_test = [X[j] for j in test_split[zzz]]
y_test = [y[j] for j in test_split[zzz]]
y_pred = [tagger.tag(xseq) for xseq in X_test]
for iterr, X_test_item in enumerate(X_test):
    for idx,word in enumerate(X_test_item):
        print(word[1]+" - "+y_test[iterr][idx]+" - "+y_pred[iterr][idx])
truth = np.array([labels[bio] for sentence in y_test for bio in sentence])
prediction = np.array([labels[bio] for sentence in y_pred for bio in sentence])
print(prediction)
print(classification_report(truth, prediction, target_names=target_names))

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 5300
Seconds required: 0.018

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3109.226570
Feature norm: 1.000000
Error norm: 2000.439133
Active features: 5194
Line search trials: 1
Line search step: 0.000125
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 2706.487574
Feature norm: 1.330529
Error norm: 718.390491
Active features: 4953
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 2605.169355
Feature norm: 1.450302
Error norm: 672.463576
Active features: 5128
Line search trials: 1
Line search step: 1.000000
Seconds required for this iterat

***** Iteration #61 *****
Loss: 442.292369
Feature norm: 62.154169
Error norm: 8.188260
Active features: 1671
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #62 *****
Loss: 441.999424
Feature norm: 62.207495
Error norm: 5.891971
Active features: 1666
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #63 *****
Loss: 441.758990
Feature norm: 62.335156
Error norm: 22.999366
Active features: 1655
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #64 *****
Loss: 441.435182
Feature norm: 62.379931
Error norm: 15.442752
Active features: 1653
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #65 *****
Loss: 441.146085
Feature norm: 62.462147
Error norm: 12.303275
Active features: 1640
Line search trials: 1
Line search step: 1.000000
Seconds required for this 

***** Iteration #145 *****
Loss: 433.923725
Feature norm: 65.038754
Error norm: 11.599513
Active features: 1458
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #146 *****
Loss: 433.846789
Feature norm: 65.023137
Error norm: 6.795264
Active features: 1457
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #147 *****
Loss: 433.822736
Feature norm: 65.042432
Error norm: 9.452244
Active features: 1462
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #148 *****
Loss: 433.766875
Feature norm: 65.030712
Error norm: 5.841939
Active features: 1462
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #149 *****
Loss: 433.730385
Feature norm: 65.045274
Error norm: 7.595557
Active features: 1464
Line search trials: 1
Line search step: 1.000000
Seconds required for th

word.lower=came - O - O
word.lower=here - O - O
word.lower=especially - O - O
word.lower=to - O - O
word.lower=enjoy - O - O
word.lower=their - O - O
word.lower=freaky - O - O
word.lower=shake - B - O
word.lower=. - O - O
word.lower=i - O - O
word.lower=came - O - O
word.lower=during - O - O
word.lower=brunch - O - O
word.lower=time - O - O
word.lower=on - O - O
word.lower=saturday - O - O
word.lower=so - O - O
word.lower=they - B - O
word.lower=are - O - O
word.lower=pretty - O - O
word.lower=busy - O - O
word.lower=and - O - O
word.lower=i - O - O
word.lower=had - O - O
word.lower=to - O - O
word.lower=put - O - O
word.lower=my - O - O
word.lower=name - O - O
word.lower=in - O - O
word.lower=the - O - O
word.lower=waiting - O - O
word.lower=list - O - O
word.lower=. - O - O
word.lower=i - O - O
word.lower=ordered - O - O
word.lower=their - O - O
word.lower=original - O - B
word.lower=cronut - O - I
word.lower=shake - O - O
word.lower=. - O - O
word.lower=the - O - B
word.lower=cronut